In [0]:
from pyspark.sql.utils import AnalysisException

start_task = "job_logging_start"
#  All actual task keys from your pipeline (excluding the logging tasks)
all_tasks = [
    "FetchingPropertyAPI",
    "Bronze_Layer_Property",
    "check_for_property_data",
    "silver_layer_property",
    "move_to_archive"
]
debug_job_id = "unknown_job_id"

# --------------------------
# STEP 1: Get job_id from start task
# --------------------------
try:
    job_id = dbutils.jobs.taskValues.get(
        taskKey=start_task,
        key="job_id",
        debugValue=debug_job_id
    )
except Exception as e:
    job_id = debug_job_id

# --------------------------
# STEP 2: Collect error messages from each task
# --------------------------
errors = []

for task in all_tasks:
    try:
        error = dbutils.jobs.taskValues.get(taskKey=task, key="error", debugValue=None)
        if error:
            errors.append(f"{task}: {error}")
    except Exception as e:
        errors.append(f"{task}: Unable to retrieve error - {str(e)}")

# --------------------------
# STEP 3: Determine status and format error message
# --------------------------
if errors:
    job_status = "Failed"
    error_message = "; ".join(errors)
else:
    job_status = "Succeeded"
    error_message = None

# Escape error message for SQL safety
if error_message:
    escaped_msg = error_message.replace("'", "''")
    error_msg_sql = f"'{escaped_msg}'"
else:
    error_msg_sql = "NULL"

# --------------------------
# STEP 4: Update job_control table
# --------------------------
update_query = f"""
    UPDATE zillow.propertyextended.job_control
    SET
        job_status = '{job_status}',
        end_time = current_timestamp(),
        error_message = {error_msg_sql}
    WHERE
        job_id = '{job_id}'
"""

try:
    spark.sql(update_query)
    print(f"Job status updated for job_id = {job_id} with status = {job_status}")
    if error_message:
        print("Error details:", error_message)
except AnalysisException as sql_ex:
    print(f"SQL error while updating job_control: {sql_ex}")
except Exception as ex:
    print(f"Unexpected error while updating job_control: {ex}")
